# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 9:20AM,261613,10,CTF0011734,"Citieffe, Inc.",Released
1,Apr 24 2023 9:20AM,261613,10,CTF0011735,"Citieffe, Inc.",Released
2,Apr 24 2023 9:18AM,261612,10,38126,"Digital Brands, LLC",Released
3,Apr 24 2023 9:18AM,261612,10,38127,"Digital Brands, LLC",Released
4,Apr 24 2023 9:18AM,261612,10,38128,"Digital Brands, LLC",Released
5,Apr 24 2023 9:18AM,261612,10,38129,"Digital Brands, LLC",Released
6,Apr 24 2023 9:18AM,261612,10,38130,"Digital Brands, LLC",Released
7,Apr 24 2023 9:18AM,261612,10,38131,"Digital Brands, LLC",Released
8,Apr 24 2023 9:18AM,261612,10,38132,"Digital Brands, LLC",Released
9,Apr 24 2023 9:18AM,261612,10,38133,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,261609,Released,2
11,261610,Released,1
12,261611,Released,1
13,261612,Released,29
14,261613,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261609,NaN,2.0
261610,NaN,1.0
261611,NaN,1.0
261612,NaN,29.0
261613,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261594,8.0,3.0
261596,0.0,19.0
261599,1.0,0.0
261600,1.0,0.0
261602,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261594,8,3
261596,0,19
261599,1,0
261600,1,0
261602,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261594,8,3
1,261596,0,19
2,261599,1,0
3,261600,1,0
4,261602,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261594,8,3
1,261596,,19
2,261599,1,
3,261600,1,
4,261602,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc."
2,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC"
31,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC"
32,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc
33,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc
35,Apr 24 2023 8:57AM,261608,10,Ivaoes Animal Health
36,Apr 24 2023 8:55AM,261607,10,"Snap Medical Industries, LLC"
37,Apr 24 2023 8:50AM,261605,10,Hush Hush
62,Apr 24 2023 8:50AM,261603,10,Hush Hush
109,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc.",,2
1,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",,29
2,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC",,1
3,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc,,1
4,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc,,2
5,Apr 24 2023 8:57AM,261608,10,Ivaoes Animal Health,,1
6,Apr 24 2023 8:55AM,261607,10,"Snap Medical Industries, LLC",,1
7,Apr 24 2023 8:50AM,261605,10,Hush Hush,,25
8,Apr 24 2023 8:50AM,261603,10,Hush Hush,,47
9,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc.",2,
1,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",29,
2,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC",1,
3,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc,1,
4,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc,2,
5,Apr 24 2023 8:57AM,261608,10,Ivaoes Animal Health,1,
6,Apr 24 2023 8:55AM,261607,10,"Snap Medical Industries, LLC",1,
7,Apr 24 2023 8:50AM,261605,10,Hush Hush,25,
8,Apr 24 2023 8:50AM,261603,10,Hush Hush,47,
9,Apr 24 2023 8:48AM,261602,16,TASA USA Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc.",2,
1,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",29,
2,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC",1,
3,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc,1,
4,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc.",2.0,NaN
1,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",29.0,NaN
2,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC",1.0,NaN
3,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc,1.0,NaN
4,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 9:20AM,261613,10,"Citieffe, Inc.",2.0,0.0
1,Apr 24 2023 9:18AM,261612,10,"Digital Brands, LLC",29.0,0.0
2,Apr 24 2023 9:02AM,261611,10,"ClearSpec, LLC",1.0,0.0
3,Apr 24 2023 8:59AM,261610,10,Eye Pharma Inc,1.0,0.0
4,Apr 24 2023 8:59AM,261609,10,Eye Pharma Inc,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2354478,109.0,0.0
16,523201,1.0,1.0
18,261600,0.0,1.0
19,523190,22.0,8.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2354478,109.0,0.0
1,16,523201,1.0,1.0
2,18,261600,0.0,1.0
3,19,523190,22.0,8.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,109.0,0.0
1,16,1.0,1.0
2,18,0.0,1.0
3,19,22.0,8.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,16,Released,1.0
2,18,Released,0.0
3,19,Released,22.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19
Status,,,,
Executing,0.0,1.0,1.0,8.0
Released,109.0,1.0,0.0,22.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19
0,Executing,0.0,1.0,1.0,8.0
1,Released,109.0,1.0,0.0,22.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19
0,Executing,0.0,1.0,1.0,8.0
1,Released,109.0,1.0,0.0,22.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()